**Insights**
* The big problem I'm trying to solve here is to make the relatively complex system of NNs for NLP something I cant 'wrap my head around'
    * It's about learning, and one of the critical concepts in human learning is 'chunking'. So I've got three layers here: 
        * Top: "string of text" ==(NN system)==> username of interest [Y/N]
        * Second: Collection of scripts with input/knobs/dials/outputs
        * Third: Code itself, presented as close to linearly as possible
        * Important to not try going too deep (I made this mistake with bash tools). Once you've got some solid abstractions that give you flexibility and don't seem to leak, extend rather than deepening knowledge
    * Connections to concrete concepts that I already know are also key, so making the inputs/outputs clear in examples using Python datatypes that I'm comfortable with will help grasp the abstract transforms 
    * Reenforcement is key, so I'll try to copy this and apply it to a different problem almost as soon as I'm done
    * The problem with many other systems is that, in an admirable effort to be modular, they force you to jump 3-7 levels deep within functions to get to the actual Python code (the concrete concept). From the main script, it can take several minutes to trace how an input and parameter actually get combined into an output.
        * Even if effort was made to document, there's a "curse of knowledge" issue that makes it difficult to grasp the particulars without the mental model that the original writer had
    * Goal is to find the mental models where I can look at other people's output (code), break it down quickly and accurately, identify what's new about it, and how i could update my design process to take advantage of anything that I don't yet have
        * [This twitter thread](https://twitter.com/michael_nielsen/status/1074150124169773056) talks about meeting 'magicians' who are better in ways that you can't comprehend, then working to understand the implicit models that allow them to be 10x better. You probably can't do it just from their work, you need to communicate on a more abstract level. How do I do that here?
    * Get solid on one simple approach, and then identify ways to extend it that I don't fully understand yet. Nailing down specific questions, "I would have expected X, but I'm seeing Y - what's going on?" is critical
* There's huge value in having a single file where you can see everything, whether it's a diagram, makefile or Jupyter notebook
* Need to identify what needs to be flexible. Passing a trivial amount of data e2e should be very doable. Then building on that, so that the data is essentially flowing (not moving in massive blocks)
* From a documentation standpoint, unit tests waste too much time on the edge cases. At the very least, the top test should always be 'happy path' so you can see what it _should_ look like. Then there needs to be a connection between files
    * This is much more possible for data pipelines than application code, so it's under-developed
* One big thing that I think I lack is an intuitive sense of what's "expensive", in terms of Disk, IO, RAM and Compute (are there other limited resources?)
    * Is streaming data from a server going to be a bottleneck?
    * Is it computationally expensive to open a bunch of little data files, instead of one big one?